<a href="https://colab.research.google.com/github/ryanvuu/csc-365-lab7/blob/main/csc_365_lab7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install libraries

In [ ]:
%pip install mysql-connector-python
import getpass
import mysql.connector
from datetime import datetime, timedelta

### FR1: Rooms and Rates ###

In [ ]:
def print_rooms_rates(conn):
    with conn.cursor() as cursor:
        cursor.execute("""
                        WITH PopTable AS (
                            SELECT lab7_rooms.RoomCode,
                                lab7_rooms.RoomName,
                                lab7_rooms.Beds,
                                lab7_rooms.bedType,
                                lab7_rooms.maxOcc,
                                lab7_rooms.basePrice,
                                lab7_rooms.decor,
                                ROUND(SUM(COALESCE(DATEDIFF(lab7_reservations.Checkout, lab7_reservations.CheckIn), 0)) / 180, 2) AS PopScore
                            FROM lab7_rooms
                            LEFT JOIN lab7_reservations
                                ON lab7_rooms.RoomCode = lab7_reservations.Room
                                AND lab7_reservations.CheckIn >= DATE_SUB(CURDATE(), INTERVAL 180 DAY)
                            GROUP BY lab7_rooms.RoomCode
                        ), NextAvailable AS (
                            SELECT RoomCode,
                            CASE WHEN
                            -- If a reservation goes past the current date
                            EXISTS (
                                SELECT *
                                FROM lab7_reservations
                                WHERE lab7_reservations.Room = lab7_rooms.RoomCode
                                    AND lab7_reservations.CheckIn <= CURDATE()
                                    AND lab7_reservations.Checkout >= CURDATE()
                            ) THEN (
                                -- Get the earliest checkout date that occurred after the current date
                                SELECT MIN(first_res.Checkout)
                                FROM lab7_reservations AS first_res
                                WHERE first_res.Room = lab7_rooms.RoomCode
                                AND first_res.Checkout > CURDATE()
                                -- See if another guest checked in on the date that the prev. reservaton checked out
                                AND NOT EXISTS (
                                    SELECT *
                                    FROM lab7_reservations as second_res
                                    WHERE first_res.Room = second_res.Room
                                    AND second_res.CheckIn = first_res.Checkout
                                )
                            )
                            ELSE CURDATE()
                            END AS NextAvailableCheckIn
                            FROM lab7_rooms
                        ), RecentReservations AS (
                            SELECT RoomCode,
                            DATEDIFF(
                            (
                                SELECT MAX(Checkout)
                                FROM lab7_reservations
                                WHERE lab7_rooms.RoomCode = lab7_reservations.Room
                                    AND lab7_reservations.Checkout <= CURDATE()
                            ),
                            (
                                SELECT MAX(Checkin)
                                FROM lab7_reservations
                                WHERE lab7_rooms.RoomCode = lab7_reservations.Room
                                AND lab7_reservations.Checkout <= CURDATE() 
                            )
                            ) AS MostRecDuration,
                            (
                            SELECT MAX(Checkout)
                            FROM lab7_reservations
                            WHERE lab7_rooms.RoomCode = lab7_reservations.Room
                                AND lab7_reservations.Checkout <= CURDATE()
                            ) AS MostRecentCheckout
                            FROM lab7_rooms
                        )
                        SELECT *
                        FROM PopTable
                        NATURAL JOIN NextAvailable
                        NATURAL JOIN RecentReservations
                        ORDER BY PopScore DESC;
                        """)
        result = cursor.fetchall()
        print(result)

### FR2: Reservations ###

In [ ]:
def make_reservation(conn):
  with conn.cursor(dictionary=True) as cursor:
    first_name = input("Enter first name: ")
    last_name = input("Enter last name: ")

    if not first_name.isalpha() or not last_name.isalpha():
      print("Error: Invalid name(s), they should only include letters!")
      return

    room_code = input("Enter room code [OPTIONAL: Any]: ").upper()
    if room_code != "ANY":
      cursor.execute("SELECT COUNT(*) FROM rvu03.lab7_rooms WHERE ROOMCODE = %s", [room_code])
      output = cursor.fetchall()
      if not output: 
        print("Error: Invalid room code!")
        return

    bed_type = input("Enter bed type [OPTIONAL: Any]: ")
    if bed_type.capitalize() != "Any":
      cursor.execute("SELECT COUNT(*) FROM rvu03.lab7_rooms WHERE bedType = %s", [bed_type])
      output = cursor.fetchall()
      if not output:
        print("Error: Invalid bed type!")
        return
    try:     
      checkin = input("Enter checkin date in the following format YYYY-MM-DD: ")
      checkout = input("Enter checkout date in the following format YYYY-MM-DD: ")

      checkin = datetime.strptime(checkin, "%Y-%m-%d")
      checkout = datetime.strptime(checkout, "%Y-%m-%d")

      if checkin >= checkout:
        print("Invalid checkin and checkout times. Please try again.")
        return
    except ValueError:
        print("Invalid date format, it should be YYYY-MM-DD.")
        return
    
    try:
      num_adults = input("Enter number of adults for reservation: ")
      num_children = input("Enter number of children for reservation: ")

      if not num_adults.isdigit() or not num_children.isdigit():
        print("Error: Invalid number of adults and children must be numerics values.")
      
      num_adults = int(num_adults)
      num_children = int(num_children)

      total_guests = num_adults + num_children
      if total_guests <= 0:
        print("Error: Invalid total number of guests, you need at least one person in the room.")
    except ValueError:
      print("Invalid adults and children numbers!")

    
    query = """
      SELECT *
      FROM rvu03.lab7_rooms AS rooms
      WHERE (%s = 'ANY' OR rooms.RoomCode = %s)
        AND (%s = 'Any' OR rooms.bedType = %s)
        AND rooms.maxOcc >= %s
        AND NOT EXISTS (
          SELECT * 
          FROM rvu03.lab7_reservations AS reservations
          WHERE reservations.Room = rooms.RoomCode
          AND reservations.CheckIn < %s
          AND reservations.CheckOut > %s
        )
    """
    cursor.execute(query, [room_code, room_code, bed_type, bed_type, total_guests, checkout, checkin])
    available_rooms = cursor.fetchall()
    # print(available_rooms)

    if not available_rooms:
      print("No exact matches found, but let's check if we can get you the next best options...")
      suggestion_query = """
        WITH AlternateRooms AS (
        SELECT *
        FROM rvu03.lab7_rooms
        WHERE RoomCode NOT IN (
            SELECT reservations.Room
            FROM rvu03.lab7_reservations AS reservations
            WHERE reservations.CheckIn < %s -- checkout
              AND reservations.Checkout > %s -- checkin
          )
        ), AverageRoomCosts AS (
            SELECT 
                rvu03.lab7_rooms.RoomCode, 
                ROUND(AVG(DATEDIFF(rvu03.lab7_reservations.Checkout, rvu03.lab7_reservations.CheckIn) * rvu03.lab7_reservations.Rate), 2) AS TotalExpense
            FROM rvu03.lab7_rooms
            INNER JOIN rvu03.lab7_reservations
                ON rvu03.lab7_rooms.RoomCode = rvu03.lab7_reservations.Room
            GROUP BY rvu03.lab7_rooms.RoomCode   
        )
        SELECT *
        FROM AlternateRooms
        NATURAL JOIN AverageRoomCosts
        WHERE maxOcc >= %s
        ORDER BY TotalExpense ASC
        LIMIT 5;
      """
      cursor.execute(suggestion_query, [checkout, checkin, total_guests])
      suggestions = cursor.fetchall()

      if not suggestions:
        print("It looks like we were not able to find any matches, Please try again with another date that works best for you.")
        return
      
      print("We were able to find you some alternative rooms, do any of these options work for you...")
      for idx, room in enumerate(suggestions, start=1):
          print(f"[{idx}] Room Code: {room['RoomCode']}, Name: {room['RoomName']}, Bed Type: {room['bedType']}, Avg Cost: ${room['TotalExpense']}")
      
      while True:
        choice = input("\nIf so select the correct option #, if not these options don't work please type 'Cancel'")

        if choice.lower() == "cancel":
          print("Booking successfully cancelled, it is sad to see you go :(.")
          return
        
        if not choice.isdigit() or not (1 <= int(choice) <= len(suggestions)):
          print(f"Please enter a valid number between 1 and {len(suggestions)}")
          continue

        selected_room = suggestions[int(choice) - 1]
        break

    else: 
        print("Available Rooms:")
        for idx, room in enumerate(available_rooms, start=1):
            print(f"[{idx}] Room Code: {room['RoomCode']}, Name: {room['RoomName']}, Bed Type: {room['bedType']}, Base Price: ${room['basePrice']}")

        while True:
            choice = input("Enter the option number to book, or type 'Cancel' to return to the main menu: ").strip()
            if choice.lower() == "cancel":
                print("Booking cancelled. Returning to main menu.")
                return
            if not choice.isdigit() or not (1 <= int(choice) <= len(available_rooms)):
                print(f"Error: Please enter a number between 1 and {len(available_rooms)}.")
                continue
            selected_room = available_rooms[int(choice) - 1]
            break


    base_rate = float(selected_room['basePrice'])
    weekdays = 0
    weekends = 0
    current_date = checkin

    while current_date < checkout:
        if current_date.weekday() < 5:
            weekdays += 1
        else:
            weekends += 1
        current_date += timedelta(days=1)

    total_cost = round(weekdays * base_rate + weekends * (base_rate * 1.1), 2)

    print(f"""
            Reservation Details:
            Guest Name: {first_name} {last_name}
            Room Code: {selected_room['RoomCode']}, Room Name: {selected_room['RoomName']}, Bed Type: {selected_room['bedType']}
            Dates: {checkin.strftime('%Y-%m-%d')} to {checkout.strftime('%Y-%m-%d')}
            Adults: {num_adults}, Children: {num_children}
            Total Cost: ${total_cost}
        """)

    confirmation = input("\nDo you want to confirm this reservation? (Yes/No): ")
    if confirmation.lower() == "yes":
        cursor.execute("SELECT MAX(CODE) FROM rvu03.lab7_reservations;")
        max_code = cursor.fetchone()['MAX(CODE)'] or 0
        new_code = max_code + 1
        insert_query = """
        INSERT INTO rvu03.lab7_reservations (CODE, Room, CheckIn, Checkout, Rate, LastName, FirstName, Adults, Kids)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);
        """
        cursor.execute(insert_query, [new_code, selected_room['RoomCode'], checkin, checkout, base_rate, last_name, first_name, num_adults, num_children])
        conn.commit()
        print("Reservation successfully confirmed")
    else:
        print("The reservation was not confirmed. Cancelling the transaction and returning to the main menu...")

### FR3: Reservation Cancellation ###

In [ ]:
def cancel_reservation(conn):
  with conn.cursor() as cursor:
    print("Delete Reservation")
    print("----------")
    reservation_code = input("Enter Reservation Code: ")
    cursor = conn.cursor()
    cursor.execute("SELECT CODE, Room, CheckIn, Checkout, Rate, LastName, FirstName, Adults, Kids FROM lab7_reservations WHERE CODE = %s", [reservation_code])
    result = cursor.fetchall()
    
    if result:
      print(result)
      confirmation = input("Are you sure you want to cancel this reservation? [Y]es]")

      if confirmation in ("Y", "Yes"):
          cursor.execute("DELETE FROM lab7_reservations WHERE CODE = %s", [reservation_code])
          conn.commit()
          print("You successfully cancelled the reservation. Thank you for considering us!")
      else:
          print("The reservation was not cancelled. Thank you for your staying with us!")
    else:
        print("The reservation code was not found. Please double check your code and try again.")

### FR4: Reservation Information ###

In [ ]:
def reservation_details(conn):
  with conn.cursor() as cursor:
    print("Get Reservation Information")
    print("----------")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM lab7_reservations")
    result = cursor.fetchall()
    print(result)
    

### FR5: Revenue ###

In [ ]:
def check_revenue(conn):
  with conn.cursor() as cursor:
    print("Check revenue")
    print("----------")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM lab7_reservations")
    result = cursor.fetchall()
    print(result)
    


# **Main Program**

In [ ]:
def main():
    try:
        db_password = getpass.getpass()
        conn = mysql.connector.connect(user='delaluz', password=db_password,
                               host='mysql.labthreesixfive.com',
                               database='delaluz')
        cursor = conn.cursor()
        while True:
            print("Select a Reservation Option:")
            print("[1] Rooms and Rates")
            print("[2] Reservations")
            print("[3] Reservation Cancellation")
            print("[4] Detailed Reservation Information")
            print("[5] Revenue")
            print("[6] Quit")

            choice = input()
            if choice == "1":
                print_rooms_rates(conn)
            elif choice == "2":
                make_reservation(conn)
            elif choice == "3":
                cancel_reservation(conn)
            elif choice == "4":
                reservation_details(conn)
            elif choice == "5":
                check_revenue(conn)
            elif choice == "6":
                break
    finally:
        cursor.close()
        conn.close()

In [ ]:
if __name__ == "__main__":
    main()